In [12]:
import PyPDF2
import spacy
from spacy.matcher import Matcher

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

nlp = spacy.load("en_core_web_sm")

def analyze_resume(text):
    doc = nlp(text)
    
    # Extract named entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    # Extract skills (nouns and proper nouns)
    skills = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN"]]
    
    # Extract key phrases (noun chunks)
    key_phrases = [chunk.text for chunk in doc.noun_chunks]
    
    # Extract responsibilities
    matcher = Matcher(nlp.vocab)
    responsibility_patterns = [
        [{"POS": "VERB"}, {"POS": "NOUN"}],
        [{"POS": "VERB"}, {"POS": "ADP"}, {"POS": "NOUN"}],
        [{"POS": "VERB"}, {"POS": "DET"}, {"POS": "NOUN"}],
    ]
    matcher.add("RESPONSIBILITY", responsibility_patterns)
    matches = matcher(doc)
    responsibilities = [doc[start:end].text for _, start, end in matches]
    
    return {
        "entities": entities,
        "skills": list(set(skills)),  # Remove duplicates
        "key_phrases": list(set(key_phrases)),  # Remove duplicates
        "responsibilities": list(set(responsibilities))  # Remove duplicates
    }

# Usage
pdf_path = "cv.pdf"
resume_text = extract_text_from_pdf(pdf_path)
parsed_data = analyze_resume(resume_text)

print(parsed_data)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.